# CAD Prompt Enhancer

This notebook takes a user's CAD generation prompt, enriches it with manufacturing knowledge through a series of LLM calls and knowledge base lookups, and then outputs a more detailed and manufacturable prompt for a CAD generation model.

## 1. User Inputs for LLM Configuration

At each step, you can choose the LLM model you prefer by providing the API URL, request structure, API key, model name, and temperature.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

style = {'description_width': 'initial'}

deconstruction_llm_url = widgets.Text(description='Deconstruction LLM API URL:', style=style, layout=widgets.Layout(width='50%'))
deconstruction_llm_key = widgets.Password(description='Deconstruction LLM API Key:', style=style, layout=widgets.Layout(width='50%'))
deconstruction_llm_model = widgets.Text(description='Deconstruction LLM Model:', style=style, layout=widgets.Layout(width='50%'))
deconstruction_llm_temp = widgets.FloatSlider(description='Deconstruction LLM Temp:', min=0, max=2, step=0.1, value=0.7, style=style)

requirements_llm_url = widgets.Text(description='Requirements LLM API URL:', style=style, layout=widgets.Layout(width='50%'))
requirements_llm_key = widgets.Password(description='Requirements LLM API Key:', style=style, layout=widgets.Layout(width='50%'))
requirements_llm_model = widgets.Text(description='Requirements LLM Model:', style=style, layout=widgets.Layout(width='50%'))
requirements_llm_temp = widgets.FloatSlider(description='Requirements LLM Temp:', min=0, max=2, step=0.1, value=0.7, style=style)

inspiration_llm_url = widgets.Text(description='Inspiration LLM API URL:', style=style, layout=widgets.Layout(width='50%'))
inspiration_llm_key = widgets.Password(description='Inspiration LLM API Key:', style=style, layout=widgets.Layout(width='50%'))
inspiration_llm_model = widgets.Text(description='Inspiration LLM Model:', style=style, layout=widgets.Layout(width='50%'))
inspiration_llm_temp = widgets.FloatSlider(description='Inspiration LLM Temp:', min=0, max=2, step=0.1, value=0.7, style=style)

display(deconstruction_llm_url, deconstruction_llm_key, deconstruction_llm_model, deconstruction_llm_temp)
display(requirements_llm_url, requirements_llm_key, requirements_llm_model, requirements_llm_temp)
display(inspiration_llm_url, inspiration_llm_key, inspiration_llm_model, inspiration_llm_temp)

## 2. Initial CAD Prompt

In [ ]:
initial_prompt = widgets.Textarea(
    description='Enter your initial CAD prompt:',
    layout=widgets.Layout(width='100%', height='100px')
)
display(initial_prompt)

## 3. Prompt Deconstruction

The first LLM call deconstructs the prompt into intent, use case, and manufacturing process.

In [ ]:
import json
import requests

def deconstruct_prompt(api_url, api_key, model, temperature, prompt):
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json',
    }
    data = {
        'model': model,
        'temperature': temperature,
        'messages': [
            {
                'role': 'system',
                'content': 'You are an expert in manufacturing and CAD design. Your task is to deconstruct the given prompt into three parts: intent, use case, and manufacturing process. Respond with a JSON object with the keys \"intent\", \"use_case\", and \"manufacturing_process\".'
            },
            {
                'role': 'user',
                'content': prompt
            }
        ]
    }
    try:
        response = requests.post(api_url, headers=headers, data=json.dumps(data))
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.json()['choices'][0]['message']['content']
    except requests.exceptions.RequestException as e:
        return f"An error occurred: {e}"

deconstruction_button = widgets.Button(description="Deconstruct Prompt")
deconstruction_output = widgets.Output()

def on_deconstruction_button_clicked(b):
    with deconstruction_output:
        deconstruction_output.clear_output()
        print("Deconstructing prompt...")
        result = deconstruct_prompt(
            deconstruction_llm_url.value,
            deconstruction_llm_key.value,
            deconstruction_llm_model.value,
            deconstruction_llm_temp.value,
            initial_prompt.value
        )
        print("Deconstruction complete!")
        global deconstructed_prompt
        deconstructed_prompt = json.loads(result)
        print(json.dumps(deconstructed_prompt, indent=2))

deconstruction_button.on_click(on_deconstruction_button_clicked)
display(deconstruction_button, deconstruction_output)

## 4. Knowledge Base Integration

The deconstructed prompt is used to query a knowledge base for relevant manufacturing information.

In [ ]:
knowledge_base = [
  {
    "tool": "3D Printer",
    "design_requirements": {
      "hardware": "Print bed, extruder, stepper motors, control board",
      "software": "Slicing software (e.g., Cura, PrusaSlicer), CAD for model design",
      "materials": "Filaments (PLA, ABS, PETG), resins for SLA printers"
    },
    "limitations": {
      "size": "Limited by build volume (typically 200-300mm per dimension for desktop models)",
      "speed": "Slow for complex or large parts (hours to days)",
      "strength": "Printed parts often weaker than machined or molded parts",
      "precision": "Layer resolution (0.1-0.3mm for FDM, 0.01-0.05mm for SLA)"
    },
    "use_cases": [
      "Prototyping (product design, concept models)",
      "Custom parts (hobbyist projects, small-scale production)"
    ],
    "examples": [
      "Creality Ender 3 (FDM, affordable, hobbyist use)",
      "Formlabs Form 4 (SLA, high precision, dental/engineering)"
    ]
  },
  {
    "tool": "CNC Milling Machine",
    "design_requirements": {
      "hardware": "Spindle, cutting tools, worktable, CNC controller",
      "software": "CAD/CAM (Fusion 360, Mastercam), G-code generation",
      "materials": "Metals, plastics, wood, composites"
    },
    "limitations": {
      "size": "Restricted by machine bed size and tool reach",
      "cost": "High initial investment ($5,000-$500,000)",
      "complexity": "Requires skilled programming and setup",
      "material_waste": "Subtractive process generates scrap"
    },
    "use_cases": [
      "Precision parts (aerospace, automotive)",
      "Molds and dies (manufacturing)"
    ],
    "examples": [
      "Haas VF-2 (industrial, versatile, metal milling)",
      "Tormach 1100MX (small shop, hobbyist/professional)"
    ]
  },
  {
    "tool": "Lathe",
    "design_requirements": {
      "hardware": "Chuck, spindle, tool rest, tailstock",
      "software": "Manual or CNC control (e.g., Mach3 for CNC lathes)",
      "materials": "Metals, plastics, wood"
    },
    "limitations": {
      "geometry": "Primarily for cylindrical or symmetrical parts",
      "size": "Limited by swing and bed length",
      "skill": "Manual lathes require significant operator expertise"
    },
    "use_cases": [
      "Shafts and bushings (mechanical components)",
      "Thread cutting (bolts, screws)"
    ],
    "examples": [
      "Grizzly G0602 (manual, small shop use)",
      "Mazak QT-250 (CNC, high-volume production)"
    ]
  },
  {
    "tool": "Laser Cutter",
    "design_requirements": {
      "hardware": "Laser source (CO2, fiber), motion system, exhaust",
      "software": "Vector design (Inkscape, CorelDRAW), control software",
      "materials": "Wood, acrylic, leather, some metals (fiber lasers)"
    },
    "limitations": {
      "thickness": "Limited cutting depth (e.g., 10-20mm for CO2 on wood)",
      "material": "Not all materials safe (e.g., PVC releases toxic fumes)",
      "speed": "Slower for thick or dense materials"
    },
    "use_cases": [
      "Signage and art (custom designs, engravings)",
      "Prototyping (flat components, enclosures)"
    ],
    "examples": [
      "Glowforge Pro (CO2, hobbyist/small business)",
      "Trotec Speedy 400 (industrial, high-speed)"
    ]
  },
  {
    "tool": "Waterjet Cutter",
    "design_requirements": {
      "hardware": "High-pressure pump, nozzle, abrasive system, CNC table",
      "software": "CAD/CAM for path programming",
      "materials": "Metals, stone, glass, composites"
    },
    "limitations": {
      "cost": "High operational cost (abrasives, water treatment)",
      "speed": "Slower than laser for thin materials",
      "mess": "Requires water and abrasive cleanup"
    },
    "use_cases": [
      "Thick material cutting (steel, granite)",
      "Heat-sensitive materials (no thermal distortion)"
    ],
    "examples": [
      "OMAX 5555 (versatile, industrial use)",
      "Flow Mach 500 (high-precision, large-scale)"
    ]
  },
  {
    "tool": "Plasma Cutter",
    "design_requirements": {
      "hardware": "Plasma torch, power supply, CNC or handheld system",
      "software": "CAD/CAM for CNC plasma tables",
      "materials": "Conductive metals (steel, aluminum)"
    },
    "limitations": {
      "material": "Limited to conductive materials",
      "precision": "Lower than laser or waterjet (kerf ~1-3mm)",
      "heat": "Causes heat-affected zones"
    },
    "use_cases": [
      "Metal fabrication (structural steel, art)",
      "Automotive repair (cutting rusted parts)"
    ],
    "examples": [
      "Hypertherm Powermax 45 (handheld, small shop)",
      "CNC Plasma Table with Torchmate (industrial)"
    ]
  },
  {
    "tool": "Injection Molding Machine",
    "design_requirements": {
      "hardware": "Mold, injection unit, clamping unit",
      "software": "CAD for mold design, process control software",
      "materials": "Thermoplastics, some thermosets"
    },
    "limitations": {
      "cost": "High mold cost ($1,000-$100,000)",
      "setup": "Long lead times for mold creation",
      "volume": "Best for high-volume production"
    },
    "use_cases": [
      "Mass production (consumer goods, electronics)",
      "Complex plastic parts (medical devices)"
    ],
    "examples": [
      "Arburg Allrounder (versatile, industrial)",
      "Boy 22A (small-scale, precise)"
    ]
  },
  {
    "tool": "Drill Press",
    "design_requirements": {
      "hardware": "Drill bit, spindle, table, depth stop",
      "software": "None or basic CNC for automated models",
      "materials": "Metals, wood, plastics"
    },
    "limitations": {
      "function": "Limited to drilling and boring",
      "precision": "Dependent on operator skill (manual models)",
      "size": "Restricted by throat depth and table size"
    },
    "use_cases": [
      "Hole-making (furniture, metalwork)",
      "Simple prototyping (basic assemblies)"
    ],
    "examples": [
      "WEN 4208T (benchtop, hobbyist)",
      "Jet JDP-20MF (floor-standing, professional)"
    ]
  }
]

def query_knowledge_base(process):
    process_name = process.lower().replace('_', ' ').replace(' ', '')
    for tool in knowledge_base:
        if tool["tool"].lower().replace(' ', '') == process_name:
            return tool
    return "Process not found in knowledge base."

knowledge_base_button = widgets.Button(description="Query Knowledge Base")
knowledge_base_output = widgets.Output()

def on_knowledge_base_button_clicked(b):
    with knowledge_base_output:
        knowledge_base_output.clear_output()
        manufacturing_process = deconstructed_prompt.get('manufacturing_process', '').lower().replace(' ', '_')
        print(f"Querying knowledge base for '{manufacturing_process}'...")
        global knowledge_base_context
        knowledge_base_context = query_knowledge_base(manufacturing_process)
        print(json.dumps(knowledge_base_context, indent=2))

knowledge_base_button.on_click(on_knowledge_base_button_clicked)
display(knowledge_base_button, knowledge_base_output)

## 5. Requirements Generation

The second LLM call uses the knowledge base context to generate proper requirements.

In [ ]:
def generate_requirements(api_url, api_key, model, temperature, prompt, context):
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json',
    }
    data = {
        'model': model,
        'temperature': temperature,
        'messages': [
            {
                'role': 'system',
                'content': 'You are an expert in manufacturing and CAD design. Your task is to generate a detailed list of requirements for a CAD model based on the user\'s prompt and the provided knowledge base context. Refer to the book "Structures: Or Why Things Don''t Fall" for structural engineering principles. Respond with a JSON object with a single key \"requirements\" which is a list of strings.'
            },
            {
                'role': 'user',
                'content': f"Prompt: {prompt}\n\nKnowledge Base Context: {context}"
            }
        ]
    }
    try:
        response = requests.post(api_url, headers=headers, data=json.dumps(data))
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.json()['choices'][0]['message']['content']
    except requests.exceptions.RequestException as e:
        return f"An error occurred: {e}"

requirements_button = widgets.Button(description="Generate Requirements")
requirements_output = widgets.Output()

def on_requirements_button_clicked(b):
    with requirements_output:
        requirements_output.clear_output()
        print("Generating requirements...")
        result = generate_requirements(
            requirements_llm_url.value,
            requirements_llm_key.value,
            requirements_llm_model.value,
            requirements_llm_temp.value,
            initial_prompt.value,
            knowledge_base_context
        )
        print("Requirements generation complete!")
        global requirements
        requirements = json.loads(result)
        print(json.dumps(requirements, indent=2))

requirements_button.on_click(on_requirements_button_clicked)
display(requirements_button, requirements_output)

## 6. JSON Structuring

The requirements are structured into a JSON format.

In [ ]:
json_button = widgets.Button(description="Structure as JSON")
json_output = widgets.Output()

def on_json_button_clicked(b):
    with json_output:
        json_output.clear_output()
        print("Structuring requirements as JSON...")
        global structured_requirements
        structured_requirements = {
            "intent": deconstructed_prompt.get('intent'),
            "use_case": deconstructed_prompt.get('use_case'),
            "manufacturing_process": deconstructed_prompt.get('manufacturing_process'),
            "requirements": requirements.get('requirements')
        }
        print(json.dumps(structured_requirements, indent=2))

json_button.on_click(on_json_button_clicked)
display(json_button, json_output)

## 7. Inspiration Generation (Multimodal LLM)

The third LLM call (multimodal) generates a few parts of the entire design as inspiration.

In [ ]:
def generate_inspiration(api_url, api_key, model, temperature, prompt):
    # This is a placeholder for a real multimodal LLM call.
    # In a real scenario, this would return images or 3D model files.
    print(f"Calling multimodal LLM at {api_url} with model {model} and temperature {temperature}")
    return ["inspiration_image_1.png", "inspiration_image_2.png"] # Placeholder

inspiration_button = widgets.Button(description="Generate Inspiration")
inspiration_output = widgets.Output()

def on_inspiration_button_clicked(b):
    with inspiration_output:
        inspiration_output.clear_output()
        print("Generating inspiration...")
        global inspiration
        inspiration = generate_inspiration(
            inspiration_llm_url.value,
            inspiration_llm_key.value,
            inspiration_llm_model.value,
            inspiration_llm_temp.value,
            json.dumps(structured_requirements)
        )
        print("Inspiration generation complete!")
        print(inspiration)

inspiration_button.on_click(on_inspiration_button_clicked)
display(inspiration_button, inspiration_output)

## 8. Final CAD Prompt Generation

The final, enriched prompt is generated for the CAD generation model.

In [ ]:
final_prompt_button = widgets.Button(description="Generate Final Prompt")
final_prompt_output = widgets.Output()

def on_final_prompt_button_clicked(b):
    with final_prompt_output:
        final_prompt_output.clear_output()
        print("Generating final prompt...")
        global final_prompt
        final_prompt = {
            "structured_requirements": structured_requirements,
            "inspiration": inspiration
        }
        print(json.dumps(final_prompt, indent=2))

final_prompt_button.on_click(on_final_prompt_button_clicked)
display(final_prompt_button, final_prompt_output)